## Decimal-Binary Conversion
In Python we have the function `bin(a)` to convert an integer `a` to a string of bits, e.g. `bin(7)` equals `"0b111"`. (We also have the option of a formated string like `f"{a:b}"` to not print the prefix `"0b"`). Conversely, to convert a bit string to a decimal integer, we have `int(a_bstr, 2)`, where `a_bstr` can be one of the following (say, `a` equals `7`)
- `"0b111"`
- `"111"`

How do we do similar things in Julia? 

### Expressing Integers
The following are three ways to express the same integer `7`.

In [1]:
7

7

In [2]:
0b111

0x07

In [3]:
0x07

0x07

### Integer to Bit String
Say, we want to carry `-100` to its binary form (`32` bits).
> One can use `bitstring()`.

In [56]:
bitstring(-100)

"1111111111111111111111111111111111111111111111111111111110011100"

In [57]:
length(ans)

64

But we have to take care of the type. By default, integer literals are of type `Int64`.

In [5]:
typeof(-100)

Int64

In [7]:
bitstring(Int32(-100))

"11111111111111111111111110011100"

In [8]:
length(ans)

32

### Bit String to Integer
Now, how do we carry this bit string back into an integer?
- `parse(type, str; base)`

In [9]:
parse(Int32, bitstring(Int32(-100)); base=2)

LoadError: [91mOverflowError: overflow parsing "11111111111111111111111110011100"[39m

**(?)** Why is there a problem of overflow? Isn't there just `32` bits and isn't the first bit the signed bit (two's complement)?<br>
**(R)** Read on.

In [10]:
"1"^32

"11111111111111111111111111111111"

In [11]:
length(ans)

32

In [12]:
parse(Int32, "0"^32; base=2)

0

In [13]:
parse(Int32, "0"^1000; base=2)

0

In [14]:
parse(Int32, "0"^1; base=2)

0

In [15]:
parse(Int32, "1"^31; base=2)

2147483647

In [16]:
ans == 2^31 - 1

true

In [17]:
typemax(Int32)

2147483647

In [18]:
"1" * "0"^31

"10000000000000000000000000000000"

In [19]:
length(ans)

32

In [20]:
parse(Int32, "1" * "0"^31; base=2)

LoadError: [91mOverflowError: overflow parsing "10000000000000000000000000000000"[39m

In [21]:
parse(Int32, "-" * "1"^31; base=2)

-2147483647

In [22]:
parse(Int32, "-" * "0" * "1"^31; base=2)

-2147483647

In [23]:
parse(Int32, "-" * "0"^10 * "1"^31; base=2)

-2147483647

In [24]:
parse(Int32, "-1" * "0"^31; base=2)

-2147483648

In [25]:
parse(Int32, "-" * "1"^32; base=2)

LoadError: [91mOverflowError: overflow parsing "-11111111111111111111111111111111"[39m

In [26]:
parse(Int32, "-1"; base=2)

-1

In [27]:
parse(Int32, "-10"; base=2)

-2

In [28]:
parse(Int32, "-0010"; base=2)

-2

In [29]:
parse(Int32, "-" * "0"^1000 * "10"; base=2)

-2

In [30]:
parse(Int32, "0"; base=2)

0

In [31]:
parse(Int32, "1"; base=2)

1

In [32]:
parse(Int32, "0"^1000 * "1"; base=2)

1

In [33]:
parse(Int32, "10"; base=2)

2

Making into a table, this looks like

| Julia bit string | Decimal/Code |
| ---------------- | ------- |
| `  1111111111111111111111111111111` | `2^31 - 1` |
| `  1111111111111111111111111111110` | `2^31 - 2` |
| `  0000000000000000000000000000000` | `0` |
| `-00000000000000000000000000000001` | `-1` |
| `-00000000000000000000000000000010` | `-2` |
| `-01111111111111111111111111111111` | `-2^31 + 1` |
| `-11111111111111111111111111111111` | `-2^31` |

Now, how do we handle negative `Int32`s whose leading two bits are both `1`'s? `Peter Cheng` from the Facebook group "Julia Taiwan" helped answer this question.
> Must first parse into `Uint32` and then `reinterpret` the resulting `UInt32` back into `Int32`. Cf. the next cell.

In [34]:
reinterpret(Int32, parse(UInt32, bitstring(Int32(-100)); base=2))

-100

In [35]:
parse(UInt32, bitstring(Int32(-100)); base=2)

0xffffff9c

In [36]:
typeof(ans)

UInt32

In [37]:
parse(Int64, bitstring(Int32(-100)); base=2)

4294967196

In [38]:
typeof(ans)

Int64

Note that we can `reinterpret` an `UInt32` back into `Int32`, but we cannot do this cast by using `Int32(a)`, where `a ∈ UInt32`:

In [39]:
Int32(parse(UInt32, bitstring(Int32(-100)); base=2))

LoadError: [91mInexactError: check_top_bit(Int32, 4294967196)[39m

In [40]:
parse(UInt32, bitstring(Int32(-100)); base=2)

0xffffff9c

**(?)** How do we find the corresponding decimal integer for a hexadecimal?<br>
**(R)** `Int()` can help.

In [41]:
Int(ans)

4294967196

In [42]:
parse(UInt32, bitstring(Int32(-100)); base=2) ==
parse(Int64, bitstring(Int32(-100)); base=2)

true

In [43]:
parse(UInt32, bitstring(Int32(-100)); base=2) ===
parse(Int64, bitstring(Int32(-100)); base=2)

false

Recall that `===` tests for both
- value
- type

In [44]:
Int32(0) === Int32(1)

false

In [45]:
typeof(parse(UInt32, bitstring(Int32(-100)); base=2)),
typeof(parse(Int64, bitstring(Int32(-100)); base=2))

(UInt32, Int64)

Note that the following scheme doesn't work
- First `parse` into `Int64` instead of into `UInt32`
- Then `reinterpret` back to `Int32`

In [46]:
reinterpret(Int32, parse(Int64, bitstring(Int32(-100)); base=2))

LoadError: [91mbitcast: argument size does not match size of target type[39m

Other combinations won't work, either.

In [47]:
reinterpret(Int64, parse(Int64, bitstring(Int32(-100)); base=2))

4294967196

In [48]:
reinterpret(Int64, parse(UInt64, bitstring(Int32(-100)); base=2))

4294967196

In [49]:
subtypes(Integer)

3-element Array{Any,1}:
 Bool
 Signed
 Unsigned

In [50]:
unsigned(Int32)

UInt32

In [51]:
entier(bit_str, t) =
  reinterpret(t, parse(unsigned(t), bit_str; base=2))

entier (generic function with 1 method)

In [52]:
entier(bitstring(-100), Int32)

LoadError: [91mOverflowError: overflow parsing "1111111111111111111111111111111111111111111111111111111110011100"[39m

The error took place because `-100` above is of type `Int64` but you asked to convert to `Int32`. Maybe you need to re-implement your function `entier` to make it more friendly.

In [54]:
entier(bitstring(Int32(-100)), Int32)

-100